# Milky Way Mapper's Galaxy

## Section 7: Paper Task

In [1]:
#Import some things
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import fits, ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
%matplotlib inline


In [2]:
#load in the data (may have to change this for wherever you downloaded your file)
#in google colab you can get the file using
#!wget https://dr19.sdss.org/sas/dr19/spectro/astra/0.6.0/summary/astraAllStarASPCAP-0.6.0.fits.gz 

filename='astraAllStarASPCAP-0.6.0.fits'
tb = fits.open(filename)
header=tb[2].header
data = tb[2].data 

In [3]:
good=np.where((data['teff'] > 3700) & (data['teff'] < 5300) &
               (data['logg'] > 0.9) & (data['logg'] < 3.3) &
               (data['m_h_atm'] > -2.0) & (data['m_h_atm'] < 0.6) &
               (data['flag_bad']==False) )

data_masked=data[good]

## Training Sets

In [4]:
#TESS
tessraw = Table.read("Theodoridis2025.csv", format="ascii")
#this one has an age column in Gyr already so we're just going to rename it Age
tessraw['Final_age'].name='Age'
hasagetess=np.where((tessraw['Age']==tessraw['Age']) & (tessraw['Age']>0.1) &(tessraw['Flag']==0))
tess=tessraw[hasagetess]
tess

TIC,Star_type,Age,νmax,Radius_gaia,Teff_xgboost,M_H_xgboost,Logg_xgboost,Logg_seis,E_Logg_seis,Mass_seis,E_Mass_seis,Initial_mass,Teff_rgb,Teff_rc,Median_age_rgb,Median_age_rc,E_lower_age_rgb,E_upper_age_rgb,E_lower_age_rc,E_upper_age_rc,Teff_diff,Flag
int64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
347020604,Clump,4.933697638292032,32.2,10.8,4766.6,-0.161,2.432,2.4173752480453827,0.0366707608673319,1.1120077414967078,0.0588614326241161,1.2303580537284051,4573.560449427352,4592.554431111989,7.362373244,4.928036510093708,6.081092717,8.892029592,4.299934128259921,6.1356166056543815,174.04556888801108,0
365250045,RGB,5.919390103924665,33.0,10.8,4931.7,-0.342,2.455,2.4229857865111493,0.0357953759646522,1.126466677413624,0.056320640605,1.1264672739859862,4667.980210870163,4834.105313542321,5.948811656,4.27541564590489,5.120775603,7.010332121,3.7125897559954217,4.990341868284544,97.59468645767902,0
377058143,RGB,10.659761236332615,59.1,7.3,4732.1,-0.519,2.615,2.677206425366256,0.0648094481080465,0.9241388003056158,0.0471439463924816,0.9241384053448526,4809.243666602538,4845.904744096898,11.11556225,5.653288060595535,9.53022434,12.72313934,4.950738403099905,6.798479977799735,-113.8047440968976,0
347548024,RGB,4.2433496208314345,33.2,11.5,4634.4,-0.213,2.388,2.4218043131513705,0.0720751969086103,1.2737524436569616,0.0852469599851652,1.2737527853903323,4625.72134164397,4775.614677688445,4.196919598,3.1842561951173276,3.279495422,5.737266629,2.7003724708906987,3.926199760609352,-141.2146776884456,0
328321210,Clump,5.297976079237009,35.7,9.6,4982.0,-0.526,2.441,2.4671643779400427,0.0755464565388593,0.9853531626679212,0.0677079590734425,1.121116190405825,4737.2709053692715,4762.055765851048,9.131354832,5.481214180765681,6.619391066,11.17929356,4.57496878200937,6.826530999798273,219.9442341489521,0
328321103,RGB,3.76901613056093,34.5,11.4,4765.0,-0.277,2.533,2.438719284222579,0.0355087401732596,1.3014095644450476,0.0659512559797257,1.301409615256684,4669.883118807332,4814.012809659391,3.7183394,2.964010779639734,3.222621332,4.398228001,2.51085645776994,3.520642797490074,-49.01280965939077,0
328324062,Clump,2.303632505100647,41.3,11.3,4792.0,0.085,2.617,2.5205598637381987,0.0266418863144747,1.5438382675435354,0.0669944467135548,1.6114631436790468,4552.67752843955,4567.434071459611,2.700422808,2.2357112051063632,2.255954628,3.155200636,1.9961451072036724,2.638300639580447,224.565928540389,0
328400618,Clump,2.159728298323744,44.1,10.5,4910.2,-0.319,2.472,2.557435937807216,0.0739867137774788,1.4511074516675977,0.0926043469167527,1.5165343396975142,4767.690838544538,4778.114861413547,2.561181642,2.157582282377099,2.082416797,3.109146383,1.7850147949917456,2.6517500725099072,132.08513858645256,0
328255103,Clump,6.418514994888695,31.9,10.8,4686.6,0.019,2.433,2.406553604071016,0.0559847430029249,1.0846413740056715,0.0674371343640133,1.18744043140101,4471.252872082005,4487.596554271885,9.075683921,6.586728636664553,7.526636104,11.69375071,5.35273124007062,7.587960124712628,199.00344572811537,0


In [5]:
# # Age option 2 APOKASC-2 Pinsonneault et al. 2018
# #Reading in the table, making sure all the tables have a column named Age in Gyr
# # and that every star in the table has an Age
# apokasc2raw = Table.read("Pinsonneault2018.txt", format="ascii.cds")
# apokasc2raw['Age']=(10**np.array(apokasc2raw['LogAge'])/1000.) #Age was in log(Myr) so needs converting
# hasagea2=np.where((apokasc2raw['Age']==apokasc2raw['Age']) & (apokasc2raw['Age']>0.1))
# apokasc2=apokasc2raw[hasagea2]
# apokasc2

In [6]:
# Age option 3 APOKASC-3 Pinsonneault et al. 2025
#Reading in the table, making sure all the tables have a column named Age in Gyr
# and that every star in the table has an Age
apokasc3raw= Table.read("Pinsonneault2025.txt", format="ascii.cds")
#in this case there were two age columns, one for Red Clump and one for Red Giant Branch so we combine them
ageRC=np.array(apokasc3raw['AgeRC']*(apokasc3raw['EvolState']=='RC'))
rcnans=np.isnan(ageRC) #removing nans from this version of the table.
ageRC[rcnans]=0
ageRGB=np.array(apokasc3raw['AgeRGB']*(apokasc3raw['EvolState']=='RGB'))
rgbnans=np.isnan(ageRGB) #removing nans from this version of the table.
ageRGB[rgbnans]=0
apokasc3raw['Age']=(ageRC+ageRGB)

hasagea3=np.where((apokasc3raw['Age']==apokasc3raw['Age']) & (apokasc3raw['Age']>0.1))
apokasc3=apokasc3raw[hasagea3]
apokasc3

KIC,EvolState,ESSource,CatTab,SeisSource,SpecSource,NNumax,NDNu,Nquar,Numax,e_Numax,DNu,e_DNu,FDNu,e_FDNu,FNumax,Mass,e_Mass,Radius,e_Radius,logg-Seis,e_logg-Seis,Teff,e_Teff,logg-Spec,e_logg-Spec,[Fe/H],e_[Fe/H],[a/Fe],e_[a/Fe],[C/Fe],e_[C/Fe],[N/Fe],e_[N/Fe],InvRGaia,e_InvRGaia,AgeCat,AgeRGB,E_AgeRGB,e_AgeRGB,AgeRC,E_AgeRC,e_AgeRC,vsini,alphaCat,GaiaDR3,2MASS,Age
,,,,,,,,,uHz,uHz,uHz,uHz,,,,Msun,Msun,Rsun,Rsun,dex(cm / s2),dex(cm / s2),K,K,dex(cm / s2),dex(cm / s2),dex(---),dex(---),dex(---),dex(---),dex(---),dex(---),dex(---),dex(---),,,,Gyr,Gyr,Gyr,Gyr,Gyr,Gyr,km / s,,,,
int64,str7,str4,str8,str4,str4,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str7,float64,float64,float64,float64,float64,float64,float64,str5,int64,str23,float64
893214,RGB,Seis,Gold,WAvg,DR17,8,5,15,40.5841,0.2874,4.3254,0.0289,1.0277,0.005,0.9976,1.4404,0.0602,11.0014,0.2055,2.5146,0.0042,4718.9233,44.7811,2.4559,0.058,-0.2617,0.058,0.0815,0.022,-0.0626,0.0135,0.2483,0.0164,0.082389,0.002866,RGB,2.8815,0.298,-0.2639,2.8815,0.298,-0.2639,0.0,Apoor,2050237616959273728,2MASS J19245967+3638183,2.8815
1026180,RC,Spec,Gold,WAvg,DR17,8,5,4,35.6089,0.2338,3.9208,0.0265,0.9972,0.005,0.9936,1.5334,0.0633,12.2361,0.2278,2.4512,0.0039,4576.1016,40.5161,2.4066,0.058,0.2741,0.058,0.0215,0.022,0.0568,0.0084,0.3647,0.0107,0.085599,0.002161,RC,--,--,--,3.1325,0.2754,-0.1905,0.0,Apoor,2050237174589477888,2MASS J19241923+3645378,3.1325
1026309,RGB,Seis,Silver,WAvg,DR17,8,4,18,16.6974,0.5675,1.9432,0.0805,1.0072,0.005,1.0205,2.642,0.518,23.2678,2.0993,2.1176,0.0148,4479.2246,39.7068,2.2388,0.058,0.1609,0.058,-0.0295,0.022,-0.0823,0.0088,0.2913,0.011,0.047943,0.001239,RGB_AGB,0.5842,0.1481,-0.109,0.7082,0.1832,-0.1405,0.0,Apoor,2050236934071312384,2MASS J19242636+3643594,0.5842
1026452,RC,Seis,Gold,WAvg,DR17,8,5,18,34.3652,0.2256,3.9749,0.0268,0.9953,0.005,0.9936,1.4618,0.0599,11.9485,0.2215,2.451,0.0044,4910.6035,53.0693,2.4907,0.058,-0.2652,0.058,0.0658,0.022,0.0005,0.0189,0.1605,0.0222,0.075937,0.003795,RC,--,--,--,2.6495,0.2319,-0.1919,0.0,Apoor,2050243050104808960,2MASS J19243452+3647244,2.6495
1027110,RGB,Seis,Silver,WAvg,DR17,8,4,18,6.5198,0.1616,1.1613,0.044,1.0511,0.005,1.0535,1.0367,0.1769,23.337,1.8791,1.6949,0.011,4194.4375,37.9582,1.7495,0.058,-0.3017,0.058,0.2615,0.022,0.1451,0.0109,0.1366,0.0131,0.040245,0.001509,RGB_AGB,9.0694,8.0551,-3.8133,7.2088,4.6171,-2.3475,0.0,Arich,2050239201814200192,2MASS J19250937+3644599,9.0694
1027337,RGB,Seis,Gold,WAvg,DR17,9,6,18,74.3689,0.4491,6.9661,0.0418,1.0282,0.005,0.9959,1.2676,0.0489,7.6702,0.133,2.7732,0.0038,4621.996,41.3674,2.7836,0.058,0.2081,0.058,0.0354,0.022,0.062,0.009,0.2898,0.0115,0.123008,0.0033,RGB,5.8519,0.5953,-0.5247,5.8519,0.5953,-0.5247,0.0,Apoor,2050240782362231552,2MASS J19252021+3647118,5.8519
1027707,RGB,Seis,Silver,WAvg,DR17,6,3,18,3.0206,0.0885,0.5593,0.0309,1.0265,0.005,1.0663,2.0033,0.4792,48.0566,5.5208,1.3484,0.0129,3961.365,35.9864,1.4062,0.058,0.1105,0.058,0.0356,0.022,0.0455,0.0079,0.1751,0.0094,0.018104,0.001509,RGB_AGB,1.3052,0.6198,-0.3779,1.5494,0.4593,-0.4138,0.0,Apoor,2050240129527252096,2MASS J19253846+3646103,1.3052
1160655,RGB,Seis,Silver,WAvg,DR17,5,2,18,1.5726,0.0502,0.3698,0.024,1.0608,0.005,1.0525,1.1903,0.3305,52.0838,6.9807,1.058,0.0139,3837.788,33.9314,0.9088,0.058,-0.1446,0.058,0.1098,0.022,0.0582,0.0089,0.2023,0.0105,0.018767,0.001236,RGB_AGB,5.7073,8.7454,-2.777,5.013,5.3362,-2.0267,0.0,Arich,2050252395953454464,2MASS J19232193+3650379,5.7073


In [7]:
# # Age option 4 K2 data Warfield et al. 2024
# #Reading in the table, making sure all the tables have a column named Age in Gyr
# # and that every star in the table has an Age
# apok2raw = Table.read("Warfield2024.txt", format="ascii.cds")
# #this one has an age column in Gyr already so we're just going to rename it Age
# hasageapok2=np.where((apok2raw['Age']==apok2raw['Age']) & (apok2raw['Age']>0.1))
# apok2=apok2raw[hasageapok2]
# apok2

In [8]:
#My initial pick for training set
agedata= apokasc3
agedata2= tess

In [9]:
#Option 1 TESS Theodoridis et al. 2025
intersect2, ind_a2, ind_b2 = np.intersect1d(data_masked['tic_v8_id'],agedata2['TIC'], return_indices=True)

#Option 2 APOKASC-2 Pinsonneault et al. 2018
#intersect, ind_a, ind_b = np.intersect1d(data_masked['sdss4_apogee_id'],agedata['2MASS'], return_indices=True)

#Option 3 APOKASC-3 Pinsonneault et al. 2025
intersect, ind_a, ind_b = np.intersect1d(data_masked['gaia_dr3_source_id'],agedata['GaiaDR3'], return_indices=True)

#Option 4 APO-K2 Warfield et al. 2024
#intersect, ind_a, ind_b = np.intersect1d(data_masked['sdss4_apogee_id'],agedata['APOGEE'], return_indices=True)

print(len(ind_b), len(ind_b2))

11030 15517


In [10]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [11]:
from tensorflow import keras

## First Training Set

In [12]:
fullx = np.dstack([data_masked['teff'][ind_a],data_masked['logg'][ind_a], data_masked['m_h_atm'][ind_a],
                   data_masked['alpha_m_atm'][ind_a], data_masked['c_h'][ind_a], data_masked['n_h'][ind_a]])[0]

fully = np.dstack([agedata['Age'][ind_b]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask = np.all(np.isfinite(fullx), axis=1) & np.all(np.isfinite(fully), axis=1)
fullx, fully = fullx[mask], fully[mask]

scaling_x = np.median(fullx, axis=0)
scaling_y = np.median(fully, axis=0)

fullx, fully = fullx/scaling_x, fully/scaling_y

In [13]:
neurons_per_layer=20
layers=5
iterations=200

In [14]:

#start with an input layer
inputs = keras.Input(shape=(6,))
#now we add the Dense layers (indicating the previous layer in the brackets following the layer declaration

#change this part if you're changing the number of layers
layer1 =keras.layers.Dense(neurons_per_layer, activation='relu')(inputs)
layer2 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer1)
layer3 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer2)
layer4 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer3)
layer5 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer4)

#then the output layer YOU ALSO HAVE TO MAKE THIS MATCH YOUR NUMBER OF LAYERS
outputs = keras.layers.Dense(1)(layer5)

# then we put that all together in the Model object
model = keras.Model(inputs=inputs, outputs=outputs, name='test')
#and we can print a summary to check it all went to plan
model.summary()

Model: "test"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,841 (7.19 KB)

 Trainable params: 1,841 (7.19 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [16]:
tenpercent=len(agedata['Age'][ind_b])//10 #figure out what ten percent of this set of age data is

#last name before M
#trainbin=slice(0,-1*tenpercent-1)
#testing=slice(-1*tenpercent,-1)


#last name M or later
trainbin=slice(tenpercent+1,-1)
testing=slice(0,tenpercent)


x_train, y_train = fullx[trainbin], fully[trainbin]
x_test, y_test = fullx[testing], fully[testing]

In [17]:
model.fit(x_train, y_train, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0000e+00 - loss: 1.6555 - val_accuracy: 0.0000e+00 - val_loss: 0.7498
Epoch 2/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.1391e-04 - loss: 0.9548 - val_accuracy: 0.0000e+00 - val_loss: 0.5811
Epoch 3/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.1391e-04 - loss: 0.8484 - val_accuracy: 0.0000e+00 - val_loss: 0.5001
Epoch 4/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.1391e-04 - loss: 0.7650 - val_accuracy: 0.0000e+00 - val_loss: 0.4380
Epoch 5/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.1391e-04 - loss: 0.7072 - val_accuracy: 0.0000e+00 - val_loss: 0.4064
Epoch 6/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.1391e-04 - loss: 0.6737 - val_accuracy: 0.0000e+00 - val_loss: 0.3901
Epoch 7/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.1391e-04 - loss: 0.6458 - val_accuracy: 0.0000e+00 - val_loss: 0.3685
Epoch 8/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - a

In [18]:
predictions = model.predict(x_test)
print(len(predictions))

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1103


In [19]:
DR19x = np.dstack([data_masked['teff'],data_masked['logg'], data_masked['m_h_atm'],
                   data_masked['alpha_m_atm'], data_masked['c_h'], data_masked['n_h']])[0]
print(len(data_masked['teff']))

DR19x= DR19x/scaling_x
predictionsDR19 = model.predict(DR19x)

425631
13301/13301 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step


## Training Set 2

In [20]:
fullx2 = np.dstack([data_masked['teff'][ind_a2],data_masked['logg'][ind_a2], data_masked['m_h_atm'][ind_a2],
                   data_masked['alpha_m_atm'][ind_a2], data_masked['c_h'][ind_a2], data_masked['n_h'][ind_a2]])[0]

fully2 = np.dstack([agedata2['Age'][ind_b2]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask2 = np.all(np.isfinite(fullx2), axis=1) & np.all(np.isfinite(fully2), axis=1)
fullx2, fully2 = fullx2[mask2], fully2[mask2]

scaling_x2 = np.median(fullx2, axis=0)
scaling_y2 = np.median(fully2, axis=0)
fullx2, fully2 = fullx2/scaling_x2, fully2/scaling_y2

In [21]:
tenpercent2=len(agedata2['Age'][ind_b2])//10 #figure out what ten percent of this set of age data is

#last name before M
#trainbin=slice(0,-1*tenpercent-1)
#testing=slice(-1*tenpercent,-1)


#last name M or later
trainbin2=slice(tenpercent2+1,-1)
testing2=slice(0,tenpercent2)


x_train2, y_train2 = fullx2[trainbin2], fully2[trainbin2]
x_test2, y_test2 = fullx2[testing2], fully2[testing2]

In [22]:
model.fit(x_train2, y_train2, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 7.9917e-05 - loss: 0.4048 - val_accuracy: 0.0000e+00 - val_loss: 0.2060
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 7.9917e-05 - loss: 0.2153 - val_accuracy: 0.0000e+00 - val_loss: 0.1929
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 7.9917e-05 - loss: 0.2012 - val_accuracy: 0.0000e+00 - val_loss: 0.1858
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 7.9917e-05 - loss: 0.1954 - val_accuracy: 0.0000e+00 - val_loss: 0.1842
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 7.9917e-05 - loss: 0.1921 - val_accuracy: 0.0000e+00 - val_loss: 0.1861
Epoch 6/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 7.9917e-05 - loss: 0.1909 - val_accuracy: 0.0000e+00 - val_loss: 0.1830
Epoch 7/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 7.9917e-05 - loss: 0.1903 - val_accuracy: 0.0000e+00 - val_loss: 0.1863
Epoch 8/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - ac

In [23]:
predictions2 = model.predict(x_test2)
print(len(predictions2))

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
1551


In [24]:
DR19x2 = np.dstack([data_masked['teff'],data_masked['logg'], data_masked['m_h_atm'],
                   data_masked['alpha_m_atm'], data_masked['c_h'], data_masked['n_h']])[0]
print(len(data_masked['teff']))

DR19x2= DR19x2/scaling_x2
predictionsDR19_2 = model.predict(DR19x2)

425631
13301/13301 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step


In [25]:
import pandas as pd
#Pulling Labels and Data
apokasc3_age = predictionsDR19.flatten()
tess_age = predictionsDR19_2.flatten()
age_difference = tess_age - apokasc3_age
star_ids = data_masked['tic_v8_id']
#Creating DataFrame of the results
dataframe = pd.DataFrame({
	"TIC": star_ids,
	'TESS_predicted_Gyr': tess_age,
	'APOKASC3_predicted_Gyr': apokasc3_age,
    'TESS_APOKASC3_Diff': age_difference
})
#Saving DataFrame to CSV
dataframe.to_csv('TESS_APOKASC3_Comparison.csv', index=False)

In [26]:
print(f'Average difference between data set is:', np.mean(age_difference))

Average difference between data set is: -0.07834965
